In [1]:
import sys
import os
import torch
from huggingface_hub import snapshot_download
from IPython.display import display, Audio

/home/sxk230060/miniconda3/envs/ltx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = '/home/sxk230060/TI2AV/misc/ltx_lora_training_i2v_t2v/more_scripts/'
sys.path.append(path)
from auffusion_helper import (load_wav, normalize_spectrogram, get_mel_spectrogram_from_audio, 
                              pad_spec, normalize, image_add_color, torch_to_pil, denormalize_spectrogram)
from auffusion_vocoder import Generator

In [3]:
audio_path = '/home/sxk230060/TI2AV/misc/ltx_lora_training_i2v_t2v/outputs/temp/cat_sample.wav'
# width_start, width = 0, 160
dtype = torch.float16
device = "cuda"

In [4]:
pretrained_model_name_or_path = "auffusion/auffusion-full-no-adapter"
if not os.path.isdir(pretrained_model_name_or_path):
    pretrained_model_name_or_path = snapshot_download(pretrained_model_name_or_path) 

Fetching 17 files: 100%|██████████| 17/17 [00:00<00:00, 36906.40it/s]


In [5]:
vocoder = Generator.from_pretrained(pretrained_model_name_or_path, subfolder="vocoder")
vocoder = vocoder.to(device=device, dtype=dtype)

/home/sxk230060/miniconda3/envs/ltx/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [6]:
Audio(audio_path)

In [9]:
audio, sampling_rate = load_wav(audio_path)
audio, spec = get_mel_spectrogram_from_audio(audio)
norm_spec = normalize_spectrogram(spec)
# norm_spec = norm_spec[:,:, width_start:width_start+width]
norm_spec = pad_spec(norm_spec, 1024)
norm_spec = normalize(norm_spec) # normalize to [-1, 1], because pipeline do not normalize for torch.Tensor input

# raw_image = image_add_color(torch_to_pil(norm_spec))
# display(raw_image)

In [10]:
norm_spec.shape

torch.Size([3, 256, 1024])

In [20]:
output_spec = norm_spec
output_spec = output_spec.to(device=device, dtype=dtype)

In [ ]:
# add to audio_list
denorm_spec = denormalize_spectrogram(output_spec)
denorm_spec_audio = vocoder.inference(denorm_spec)

In [22]:
denorm_spec_audio

array([[ 0,  1,  0, ...,  2, -6, -6]], dtype=int16)

In [23]:
display(Audio(denorm_spec_audio, rate=16000))